In [1]:
from google.colab import drive
import numpy as np
import cv2, os, torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!unzip -q "/content/drive/MyDrive/tiny-imagenet-2020.zip"

replace tiny-imagenet-200/train/class_102/00063.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
All


In [4]:
from torchvision import transforms
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])

In [5]:
from torchvision.datasets import ImageFolder

path_train = 'tiny-imagenet-200/train/'
path_test = 'tiny-imagenet-200/val/'

dataset_test = ImageFolder(path_test, transform = preprocess)
dataset_train = ImageFolder(path_train, transform = preprocess)

In [6]:
dataset_train

Dataset ImageFolder
    Number of datapoints: 100000
    Root location: tiny-imagenet-200/train/
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=PIL.Image.BILINEAR)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

# Create the dataset


In [9]:
np.random.seed(1)
train_loader = DataLoader(dataset=dataset_train, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=dataset_test, shuffle=True)

In [10]:
for x,y in train_loader:
  print(x.shape)
  break

torch.Size([64, 3, 224, 224])


# Set the model

In [14]:
import torchvision.models as models

model = models.resnet152()
model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

# Training

In [16]:
epochs = 15
model.train()
for i in range(epochs):
    for j, (x,y) in enumerate(train_loader):
        optimizer.zero_grad()
        
        y = y.cuda()
        x = x.cuda()

        y_pred = model(x)
        loss = criterion(y_pred, y.long())
        print(f"Epoch {i}\t iter {j}\t loss {loss}")
        loss.backward()
        optimizer.step()

Epoch 0	 iter 0	 loss 5.377848148345947
Epoch 0	 iter 1	 loss 5.2957682609558105
Epoch 0	 iter 2	 loss 5.387393474578857
Epoch 0	 iter 3	 loss 5.33180570602417
Epoch 0	 iter 4	 loss 5.278486728668213
Epoch 0	 iter 5	 loss 5.177891731262207
Epoch 0	 iter 6	 loss 5.404549598693848
Epoch 0	 iter 7	 loss 5.354905605316162
Epoch 0	 iter 8	 loss 5.549834728240967
Epoch 0	 iter 9	 loss 5.250171184539795
Epoch 0	 iter 10	 loss 5.240550518035889
Epoch 0	 iter 11	 loss 5.323581218719482
Epoch 0	 iter 12	 loss 5.313276290893555
Epoch 0	 iter 13	 loss 5.305760383605957
Epoch 0	 iter 14	 loss 5.296850204467773
Epoch 0	 iter 15	 loss 5.373499393463135
Epoch 0	 iter 16	 loss 5.545102596282959
Epoch 0	 iter 17	 loss 5.274930953979492
Epoch 0	 iter 18	 loss 5.494752407073975
Epoch 0	 iter 19	 loss 5.337712287902832
Epoch 0	 iter 20	 loss 5.392345428466797
Epoch 0	 iter 21	 loss 5.256580352783203
Epoch 0	 iter 22	 loss 5.41759729385376
Epoch 0	 iter 23	 loss 5.323928356170654
Epoch 0	 iter 24	 loss 5.37

KeyboardInterrupt: ignored

# Evaluation of the model

In [44]:
model.eval()
y_true = []; y_pred = []
for x, y in test_loader:
  x = x.cuda()
  y = y.cuda()
  _, pred = torch.max(model(x),1)
  y_pred.append(pred.item())
  y_true.append(int(y))

print('The accuracy of the model is:', accuracy_score(y_true, y_pred))

The accuracy of the model is: 0.2271
